In [18]:
# -*- coding: utf-8 -*-
"""
generate_invoices.py  – FINAL
• Address lines 9 pt
• Extra padding (name/address, meta block)
• Thin grey dividers
• Bold guaranteed via Arial‑Bold or DejaVuSans‑Bold
• NOTE block is always TWO lines; big amount stays flush‑right
"""

import os, unicodedata, datetime as _dt
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.lib import colors
from reportlab.platypus import Table, TableStyle, Paragraph
from reportlab.lib.styles import ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont


# ────────── Font setup ──────────
def reg(alias, path):
    if os.path.exists(path):
        pdfmetrics.registerFont(TTFont(alias, path))
        return True
    return False


reg("Arial", "arial.ttf")
reg("Arial-Bold", "arialbd.ttf")
reg("DejaVuSans", "DejaVuSans.ttf")
reg("DejaVuSans-Bold", "DejaVuSans-Bold.ttf")

BASE_FONT = "Arial" if "Arial" in pdfmetrics.getRegisteredFontNames() else "DejaVuSans"
BOLD_FONT = (
    "Arial-Bold"
    if "Arial-Bold" in pdfmetrics.getRegisteredFontNames()
    else "DejaVuSans-Bold"
)
RUPEE_FONT = "DejaVuSans"  # contains ₹

# ────────── Data ──────────
transaction_ID = "4308464653"

student_name = "Chinthala Vedhitha"
teacher_name = "BHAMIDIPATI SHANMUKHA SAI"
teacher_addr1 = (
    "2-2-1136/3/A/5 NIRANJAN NIVAS APARTMENT BESIDE PAPAJI DHABA LINE"
)
teacher_city = "Musheerabad"
teacher_state = "Telangana"
teacher_pin = "500044"

company_name = "Mateira Technologies Private Limited"
company_addr1 = "29 R R Road"
company_addr2 = "Ghaziabad, Uttar Pradesh"
company_addr3 = "India – 201001"

service_amount = 459000.00
platform_fees = 585.11
invoice_date = _dt.datetime(2025, 9, 2).strftime("%d %b %Y")
invoice_teacher_no = "#20250902/032"
invoice_company_no = "#20250902/031"

RUPEE = "\u20b9"

# ────────── Layout constants ──────────
PAGE_W, PAGE_H = A4
MARGIN = 10 * mm
CONTENT_W = PAGE_W - 2 * MARGIN
RULE_MARGIN = 5 * mm
TITLE_Y = PAGE_H - 30 * mm
NAME_ADDR_GAP = 7 * mm
META_LINE_HT = 5 * mm
AMT_GAP_Y = 10 * mm

p9 = ParagraphStyle("p9", fontName=BASE_FONT, fontSize=9, leading=11)


# ────────── Helper functions ──────────
def clean(txt: str):
    unwanted = (0x00A0, 0x202F, 0x2007, 0x2060)
    return unicodedata.normalize("NFKC", txt).translate(dict.fromkeys(unwanted, 0x20))


def money(v):
    return f"{RUPEE}{v:,.2f}"


def right(c, txt, xr, y, font, size, color=colors.black):
    txt = clean(txt)
    c.setFillColor(color)
    c.setFont(font, size)
    c.drawRightString(xr, y, txt)
    c.setFillColor(colors.black)


def hline(c, y):
    c.setStrokeColor(colors.grey)
    c.setLineWidth(0.3)
    c.line(RULE_MARGIN, y, PAGE_W - RULE_MARGIN, y)
    c.setStrokeColor(colors.black)
    c.setLineWidth(1)


def make_tbl(data, widths, style):
    tbl = Table(data, colWidths=widths, hAlign="LEFT")
    tbl.setStyle(TableStyle(style))
    return tbl


def draw_tbl(c, tbl, x, y):
    _, h = tbl.wrapOn(c, CONTENT_W, 0)
    tbl.drawOn(c, x, y - h)
    return h


# ────────── Section builders ──────────
def bill_meta(c, y_top, inv_no):
    bill = [["BILL TO", ":"], [student_name, ""]]
    tbl = make_tbl(
        bill,
        [25 * mm, CONTENT_W - 25 * mm],
        [
            ("FONTNAME", (0, 0), (0, 0), BOLD_FONT),
            ("FONTSIZE", (0, 0), (0, 0), 8),
            ("FONTNAME", (0, 1), (0, 1), BOLD_FONT),
            ("FONTSIZE", (0, 1), (0, 1), 11),
        ],
    )
    h_tbl = tbl.wrapOn(c, CONTENT_W, 0)[1]
    meta_x = PAGE_W - MARGIN - 2 * mm
    block_h = max(h_tbl, 6 * META_LINE_HT)
    tbl.drawOn(c, MARGIN, y_top - h_tbl)
    y = y_top
    for head, val in (
        ("INVOICE #", inv_no),
        ("DATE", invoice_date),
        ("INVOICE DUE DATE", invoice_date),
    ):
        right(c, head, meta_x, y, BOLD_FONT, 8)
        y -= META_LINE_HT
        right(c, val, meta_x, y, BASE_FONT, 8)
        y -= META_LINE_HT
    return y_top - block_h - 5 * mm


def items_tbl(c, y, row):
    widths = [40 * mm, 64 * mm, 20 * mm, 33 * mm, 33 * mm]
    data = [["SERVICES", "DESCRIPTION", "QTY", "UNIT PRICE", "AMOUNT"], row]
    style = [
        ("FONTNAME", (0, 0), (-1, 0), BOLD_FONT),
        ("FONTSIZE", (0, 0), (-1, 0), 8),
        ("FONTNAME", (0, 1), (-1, 1), BASE_FONT),
        ("FONTSIZE", (0, 1), (-1, 1), 9),
        ("ALIGN", (2, 0), (-1, -1), "RIGHT"),
        ("LEFTPADDING", (0, 0), (-1, -1), 2),
        ("RIGHTPADDING", (0, 0), (-1, -1), 2),
    ]
    y -= draw_tbl(c, make_tbl(data, widths, style), MARGIN, y) + 10
    hline(c, y)
    return y - 8 * mm


def note_total(c, y, amt):
    rx = PAGE_W - MARGIN - 2 * mm
    # Row 1 labels
    c.setFont(BOLD_FONT, 8)
    c.drawString(MARGIN, y, "NOTE:")
    right(c, "TOTAL", rx, y, BOLD_FONT, 8)
    # Row 2 (part 1) note wrap + amount
    y -= AMT_GAP_Y
    c.setFont(BASE_FONT, 9)
    c.drawString(
        MARGIN, y, "Credit Card standard settlement transactions take up to 24 hours"
    )
    right(c, money(amt), rx, y, RUPEE_FONT, 26)
    # Row 3 (part 2) remainder of note
    y -= 4 * mm
    c.setFont(BASE_FONT, 9)
    c.drawString(MARGIN, y, "for amount settlements.")


# ────────── Invoice A: Student → Teacher ──────────
def build_teacher(fn):
    c = Canvas(fn, A4)
    c.setFont(BOLD_FONT, 36)
    c.drawString(MARGIN, TITLE_Y, "Invoice")
    hdr_x = PAGE_W - MARGIN - 2 * mm
    right(
        c,
        "Issued by Mateira Technologies Private Limited on the behalf of",
        hdr_x,
        TITLE_Y,
        BASE_FONT,
        8,
        colors.grey,
    )
    y = TITLE_Y - 8 * mm
    right(c, teacher_name, hdr_x, y, BOLD_FONT, 11)
    y -= NAME_ADDR_GAP
    for line in (teacher_addr1, f"{teacher_city}, {teacher_state} - {teacher_pin}"):
        right(c, line, hdr_x, y, BASE_FONT, 9)
        y -= 5 * mm
    hline(c, y - 3 * mm)
    y -= 9 * mm
    y = bill_meta(c, y, invoice_teacher_no)
    hline(c, y)
    y -= 10 * mm
    row = [
        Paragraph("Educational Service", p9),
        Paragraph("Service rendered by the teacher to the student", p9),
        "1",
        money(service_amount),
        money(service_amount),
    ]
    y = items_tbl(c, y, row)
    note_total(c, y, service_amount)
    c.save()
    print("✓ invoice_teacher.pdf")


# ────────── Invoice B: Student → Company ──────────
def build_company(fn):
    c = Canvas(fn, A4)
    c.setFont(BOLD_FONT, 36)
    c.drawString(MARGIN, TITLE_Y, "Invoice")
    hdr_x = PAGE_W - MARGIN - 2 * mm
    right(c, company_name, hdr_x, TITLE_Y, BOLD_FONT, 11)
    y = TITLE_Y - 8 * mm - 2 * mm  # extra gap after company name
    for line in (company_addr1, company_addr2, company_addr3):
        right(c, line, hdr_x, y, BASE_FONT, 9)
        y -= 5 * mm
    hline(c, y - 3 * mm)
    y -= 9 * mm
    y = bill_meta(c, y, invoice_company_no)
    hline(c, y)
    y -= 10 * mm
    row = [
        Paragraph("Eira.club platform fees", p9),
        Paragraph("Fees for processing payments through Eira.club app", p9),
        "1",
        money(platform_fees),
        money(platform_fees),
    ]
    y = items_tbl(c, y, row)
    note_total(c, y, platform_fees)
    c.save()
    print("✓ invoice_company.pdf")


# ────────── Build PDFs ──────────
build_teacher(f'{invoice_teacher_no.replace("/", "-")}_invoice_teacher-{transaction_ID if transaction_ID else ""}.pdf')
build_company(f'{invoice_company_no.replace("/", "-")}_invoice_company-{transaction_ID if transaction_ID else ""}.pdf')


✓ invoice_teacher.pdf
✓ invoice_company.pdf
